# Multi-Class Classification - Language Classification

This notebook implements the method presented in Goldberg's [2017] book "Neural Network Methods for Natural Language Processing". It shows the steps you need to go through in order to successfully train a classifier, and it should also, so I hope, illustrate the notational differences between Goldberg and standard machine learning literature.

$NOTE$: There is no cross-validation etc. to find optimal parameters. This is simply to show how multi-class classification works. This will be part of a tutorial session and all other concepts will be explained there.

Author: Phillip Ströbel

## Getting and cleaning the data

The data consists of downloaded Wikipedia articles (see `urls.txt`) in German, English, French, Spanish, Italian and Finnish (instead of "O" in Goldberg). The data is in HTML, so we need to some preprocessing to get the text out of it. We also restrict ourselfes to the characters from a to z in the alphabet (as described in Goldberg). In this fashion, we get rid of all the Umlauts (ä, ö, ü) and all other characters with diacritics (as, e.g., the é or ç in French). Note however, that if these characters ocurring in bigrams would probably be good features. In some way, we still keep the information "special character" by not fully deleting the character, but by replacing it by the dollar sign "\$". Furthermore, we replace all punctuation marks and digits by dollar signs as well. As such, all special characters, digits, and punctuation marks are mapped to $. The space will be replaced by an underscore "\_". We then represent each langauge by 28 characters, as is suggested by Goldberg.

### Cleaning HTML
We first strip the HTML to get only the text of the Wikipedia page.

#### Get the html files

In [1]:
import re
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import urlopen
from collections import defaultdict

seed = np.random.seed(seed=200)  # set a seed for random, so results are reproducible

article_dict = defaultdict(lambda: defaultdict(str))

regex = r'[\n ]{2,}'
pattern = re.compile(regex)

urls = open('urls.txt', 'r').readlines()

for url_index, url in enumerate(urls):
    language = url[8:10]
    doc_id = 'doc_%d' % url_index
    html = urlopen(url.strip()).read()    
    soup = BeautifulSoup(html, 'html.parser')
    raw = soup.body.get_text()  # only get text from the text body (this excludes headers and should exclude navigation bars)
    raw = re.sub(pattern, ' ', raw)  # replace multiple breaks and spaces by only one space
    raw = re.sub(r'\n', ' ', raw)  # replace every line break with a space
    article_dict[language][doc_id] = raw.lower()  # assign each text to its language and lower all uppercase characters

### Preprocessing --> prepare the text
replace special characters and digits

In [2]:
preprocessed_dict = defaultdict(lambda: defaultdict(str))

abc = r'[a-z]'
abc_pattern = re.compile(abc)

for lang, doc in article_dict.items():
    for doc, text in doc.items():
        for char in text:
            if re.match(abc_pattern, char):
                preprocessed_dict[lang][doc] += char
            elif re.match(' ', char):
                preprocessed_dict[lang][doc] += '_'
            else:
                preprocessed_dict[lang][doc] += '$'

### Count bigrams --> Feature extraction

The distribution of bigrams will be our only feature. We could extend this by taking into account other n-grams.

In [3]:
charset = 'abcdefghijklmnopqrstuvwxyz$_'  # define the character set we want to use

In [4]:
from itertools import combinations_with_replacement, permutations

def bigrams(text):
    """
    Function to extract bigrams from text and calculate their distribution
    :param text: text string
    :return: dictionary containing bigrams as keys, and the normalised count as values
    """
    combs = combinations_with_replacement(charset, 2)
    perms = permutations(charset, 2)
    bigram_dict = dict()
    
    for comb in set(list(combs) + list(perms)):
        bigram_dict[''.join(comb)] = 0
        
    doc_length = len(text)
    
    for i in range(0, len(text)-1):
        bigram = text[i] + text[i+1]
        bigram_dict[bigram] += 1
                
    for bigram, count in bigram_dict.items():
        bigram_dict[bigram] = count/doc_length

    return bigram_dict              

### Put data into pandas dataframe
The pandas dataframe allows us to conveniently represent all the data we need in one table. So let's do this. But first we need to extract the features.

In [5]:
bigram_dict_full = defaultdict(lambda: defaultdict(dict))

for lang, doc in preprocessed_dict.items():
    for doc, text in sorted(doc.items()):
        bigram_dict = bigrams(text)
        bigram_dict_full[lang][doc] = bigram_dict

In [6]:
import pandas as pd

col_names = ['y'] + sorted(bigram_dict_full['en']['doc_0'].keys())
my_df = dict()

for col in col_names:
    my_df[col] = list()
    
df = pd.DataFrame(my_df)

for lang, doc in bigram_dict_full.items():
    for key, value in doc.items():
        df_obj = value
        df_obj['y'] = lang
        df = df.append(df_obj, ignore_index=True)
        
df.head()
        

,$$,$_,$a,$b,$c,$d,$e,$f,$g,$h,...,zq,zr,zs,zt,zu,zv,zw,zx,zy,zz
0,0.070389,0.034548,0.002063,0.000317,0.000249,0.000295,0.001020,0.000136,0.000113,0.001065,...,0.0,0.0,0.000000,0.0,0.000023,0.0,0.000000,0.0,0.0,0.0
1,0.047840,0.025302,0.000809,0.000243,0.000135,0.000067,0.000890,0.000067,0.000121,0.000850,...,0.0,0.0,0.000000,0.0,0.000013,0.0,0.000013,0.0,0.0,0.0
2,0.064450,0.032426,0.000541,0.001359,0.000357,0.000392,0.000841,0.000092,0.000081,0.001106,...,0.0,0.0,0.000000,0.0,0.000127,0.0,0.000012,0.0,0.0,0.0
3,0.070677,0.029689,0.000320,0.000137,0.000412,0.000457,0.001189,0.000549,0.000046,0.000549,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
4,0.046610,0.034304,0.000789,0.000168,0.000168,0.000505,0.000751,0.000233,0.000065,0.001009,...,0.0,0.0,0.000013,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0


In [7]:
df.shape

(60, 785)

Now we split the data into the label vector \begin{equation}\mathbf{y}\end{equation} and a training data matrix \begin{equation}\mathbf{X}\end{equation}. But first, we shuffle the df and split it into a training and a test set.

Split the data into a train set and a test set

In [8]:
train = df.sample(frac=0.9, random_state=seed)
test = df.drop(train.index)

Define the different sets

In [9]:
# for training
y_train = train.y
X_train = train.drop('y', axis=1)

# for testing
y_test = test.y
X_test = test.drop('y', axis=1)

Check the shapes

In [10]:
print('Training samples shape: ', X_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test samples shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Training samples shape:  (54, 784)
Training labels shape:  (54,)
Test samples shape:  (6, 784)
Test labels shape:  (6,)


Moreover, it is necessary for many machine learning tasks to standardise the data. Our aim is for each feature to be represented by a column vector in which values have zero mean and unit variance.

In [11]:
def normalise_matrix(matrix, mean_and_std=None):
    """
    normalises the data matrix (normalise each datapoint to zero mean and unit variance.)
    :param matrix: input matrix
    :param mean_and_std: provide mean and std as tuples in list for normalisation of test data
    :return: normalised matrix and list consisting of tuples containing mean and std
    """
    normalised = np.ones(matrix.shape)
    
    if mean_and_std == None:
        
        mean_std_list = list()

        for col_index, col in enumerate(matrix):
            mean = matrix[col].mean()
            std = matrix[col].std()
            mean_std_list.append((mean, std))
            for row_index, item in enumerate(matrix[col]):
                try:
                    normalised[row_index, col_index] = (item - mean)/std
                except ZeroDivisionError:
                    normalised[row_index, col_index] = 0.0
        return normalised, mean_std_list
    else:
        for col_index, col in enumerate(matrix):
            for row_index, item in enumerate(matrix[col]):
                try:
                    mean = mean_and_std[col_index][0]
                    std = mean_and_std[col_index][1]
                    normalised[row_index, col_index] = (item - mean)/std
                except ZeroDivisionError:
                    normalised[row_index, col_index] = 0.0
        return normalised

normalise the train and test set (use values from training set for standardisation of test set). in sklearn, there are simple methods which help you with this!

In [12]:
train_norm, train_mean_std = normalise_matrix(X_train)
test_norm = normalise_matrix(X_test, train_mean_std)

We should binarise our labels, although libraries like sklearn can also deal with non-numeric data

In [13]:
from sklearn import preprocessing

lb = preprocessing.LabelBinarizer()
lb.fit(['en', 'fr', 'de', 'it', 'es', 'fi'])

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [14]:
lb.classes_

array(['de', 'en', 'es', 'fi', 'fr', 'it'], dtype='<U2')

We do this for both our training and test labels:

In [15]:
y_train = lb.transform(y_train)
y_test = lb.transform(y_test)

Labels are now one-hot encoded:

In [16]:
y_train[0]

array([0, 0, 0, 0, 0, 1])

We almost have everything now. However, we need to take care of the bias and the weight matrix. The hypothesis ŷ is given by:
\begin{equation}
\mathbf{\hat{y}}=\mathbf{x}\cdot\mathbf{W}+\mathbf{b}
\end{equation}
We can achieve this by appending 1 to each feature vector x, and the whole weight vector b to the weight matrix W. This is called the bias trick. Note that the dimensions of X_train change, and that the weight matrix W will have match the dimensions (same number of rows as X has columns).

In [17]:
bias_vector = np.ones([train_norm.shape[0], 1])
X_train = np.append(train_norm, bias_vector, axis=1)
X_train.shape

(54, 785)

In [18]:
# initialise weight matrix with small weights

np.random.seed(seed=200)

W = np.random.randn(X_train.shape[1], len(lb.classes_)) * 0.0001
#W = np.zeros([X.shape[1], len(lb.classes_)])

We see that the dimensions are right. The dot product of a specific row from X_train and the weight matrix W constitutes a forward pass and calculates the score for each class.

In [19]:
W.shape

(785, 6)

In [20]:
X_train.shape

(54, 785)

In [21]:
X_train[5:6].dot(W)

array([[-0.00165842, -0.00558468, -0.00562853, -0.00551135, -0.00191775,
         0.0010696 ]])

We see that the values for the highest score of the dot product is not the score of the true label. Our aim is to change this by implementing a support vector classifier.

In [22]:
X_train[5:6].dot(W).max(axis=1)

array([0.0010696])

In [23]:
X_train[5:6].dot(W)[0][y_train[5:6].argmax()]

-0.005511349692839673

Important: we follow kind of a naive implementation. The aim is to be able to understand what is going on!

In order to quantify how good (or how bad) our weight matrix W can predict the data in our training set, we need to implement a loss function. Here we take a go at the hinge loss, which tries to predict the correct class with a margin of at least one to all other classes (or in this case, like presented in Goldberg, to the class which does not equal the true class, but which scores highest). In my understanding, this is a one-vs-one approach (true class vs. class with highest score (but doesn't equal the true class)).

In [24]:
def hinge_loss(x, y, W, index):
    """
    Calculates the loss of a single data point by taking the prediction of the correct value and the the prediction of
    the value of next highest score, following Crammer and Singer (2001)
    :param x: sample point x as a vector
    :param y: correct label y for x as a vector
    :param W: weight matrix
    :param index: column index of data matrix X
    :return: loss
    """
    loss = 0
    y_index = y[index].argmax()
    y_value = x.dot(W)[y_index]
    y_hat_max_value = np.delete(x.dot(W), y_index).max()
    #for j in range(0, y.shape[1]):  # in case we wanted to classify against all other classes (one-vs-all) --> currently one-vs-one
        #if j == y_index:
            #continue
    loss += max(0, 1 - (y_value - y_hat_max_value))
    return loss

With matrix multiplication, we could get all the scores at once. In the following, however, we focus on an approach which takes sample by sample and calculates the loss and the gradients.

In [25]:
scores = X_train.dot(W)  # simple matrix multiplication to get all scores

In [26]:
scores

array([[-5.55398818e-04,  3.37949002e-03, -8.40643524e-05,
        -2.72327890e-03,  2.27223898e-03, -4.19751279e-03],
       [-3.02864621e-03, -8.57414552e-04, -4.44178360e-03,
        -4.11820898e-03,  3.05471255e-03,  1.43402301e-03],
       [ 2.68524769e-03,  2.54146957e-03,  1.71621466e-03,
         3.04851690e-03,  5.60148080e-04,  1.58296171e-03],
       [ 6.07786877e-03,  9.80210433e-04,  1.45588397e-03,
         4.25217812e-03, -6.79212250e-04,  8.68427653e-04],
       [-2.66124536e-03,  8.88080362e-04,  3.02651856e-04,
        -7.39309752e-04,  3.83831939e-03,  4.96306015e-06],
       [-1.65841868e-03, -5.58467584e-03, -5.62853211e-03,
        -5.51134969e-03, -1.91774985e-03,  1.06960255e-03],
       [ 6.68489444e-04, -9.52709157e-04,  6.06902896e-03,
         2.99064977e-03, -2.74790493e-03,  5.99410083e-05],
       [-1.09604347e-03,  2.34331758e-03, -1.70226576e-03,
        -3.43223327e-03,  6.39593485e-04,  3.13845305e-03],
       [ 4.01153457e-03, -1.75777546e-03,  4.767

In [127]:
def gradient(X, y, W):
    """
    compute the gradient
    :param X: data matrix (train) 
    :param y: the corresponding 
    :param W: weight matrix
    :return: loss and Jacobian dW with all gradients
    """
    dW = np.zeros(W.shape)
    
    total_loss = 0.0
    
    for index, x in enumerate(X):
        y_index = y[index].argmax()
        y_value = x.dot(W)[y_index]
        y_hat_max_value = np.delete(x.dot(W), y_index).max()
        loss = max(0, 1 - (y_value - y_hat_max_value))
        total_loss += loss
        y_hat_max_index = np.delete(x.dot(W), y_index).argmax() + 1
        if loss > 0:  # not sure whether we need this if statement
            dW[:, y_hat_max_index] += x.transpose()
            dW[:, y_index] -= x.transpose()
            
    total_loss /= X.shape[0]
    dW /= X.shape[0]
            
    return total_loss, dW
    

In [128]:
def gradient_descent(X, y, W, eta, steps):
    """
    Perform gradient descent for a number of times with a fixed learning rate eta
    :param X: data matrix
    :param y: labels
    :param W: weight matrix
    :param eta: learning rate
    :param steps: number of times gradient descent should be performed
    :return: learned representation matrix W_learned
    """
    W_learned = W.copy()
    
    for step in range(0, steps):
        loss, dW = gradient(X, y, W_learned)
        print(loss)
        W_learned = W_learned - eta * dW
        
    return W_learned
    

In [136]:
W_star = gradient_descent(X_train, y_train, W, eta=0.001, steps=100)

1.0038170867499712
0.9458004956913564
0.9029223134272765
0.8359050724406546
0.7627383071817009
0.7002533175250243
0.6363839674437564
0.5601080098980816
0.4977957071945216
0.43727328194478265
0.39969043635331086
0.37194985032759115
0.3451396585114108
0.31993737904591485
0.29542868779422654
0.2730317394318695
0.25051221058149853
0.22653748291157821
0.20419158204055793
0.18030176579112206
0.1613555932139755
0.14161625629003335
0.12595610133530682
0.11406584874399786
0.10426680552175424
0.09156041227255908
0.08373285352546704
0.07634078453206532
0.07264716918242302
0.06919024286641753
0.06600104960037959
0.06336474174747486
0.062469866350450105
0.06050885893178353
0.05983604994039867
0.05923939378809413
0.05776841144964399
0.05749401804990851
0.05721962465017302
0.05714119484868436
0.054972663633005285
0.05427993779004683
0.05406309086590985
0.05353799074850154
0.053318813705064985
0.053161296647773394
0.053161296647773394
0.053161296647773394
0.053161296647773394
0.053161296647773394
0.05

### Testing
Let's test if our learned representation of the data is any good at classifying the data in the test set. Of course we need the bias in our test set as well!

In [137]:
bias_vector_test = np.ones([X_test.shape[0], 1])
X_test = np.append(test_norm, bias_vector_test, axis=1)

In [138]:
for index, x in enumerate(X_test.dot(W_star)):
    pred = x.argmax()
    true_label = y_test[index].argmax()
    print(pred, true_label)

3 3
5 5
5 5
4 4
0 0
0 0


Not too bad! But Goldberg mentioned something about regularisation, so we should take this into account as well!

In [160]:
def gradient_reg(X, y, W, lam):
    """
    compute the gradient
    :param X: data matrix (train) 
    :param y: the corresponding 
    :param W: weight matrix
    :param lam: reguliser lambda
    :return: Jacobian dW with all gradients
    """
    dW = np.zeros(W.shape)
    
    total_loss = 0.0
    
    for index, x in enumerate(X):
        y_index = y[index].argmax()
        y_value = x.dot(W)[y_index]
        y_hat_max_value = np.delete(x.dot(W), y_index).max()
        loss = max(0, 1 - (y_value - y_hat_max_value))
        total_loss += loss
        y_hat_max_index = np.delete(x.dot(W), y_index).argmax() + 1
        if loss > 0:  # not sure whether we need this if statement
            dW[:, y_hat_max_index] += x.transpose()
            dW[:, y_index] -= x.transpose()
    
    loss /= X.shape[0]
    dW /= X.shape[0]
    
    total_loss += lam * np.linalg.norm(W, 2)
    dW += lam * W
            
    return total_loss, dW

def gradient_descent_reg(X, y, W, eta, steps):
    """
    Perform gradient descent for a number of times with a fixed learning rate eta
    :param X: data matrix
    :param y: labels
    :param W: weight matrix
    :param eta: learning rate
    :param steps: number of times gradient descent should be performed
    :return: learned representation matrix W_learned
    """
    W_learned = W.copy()
    
    for step in range(0, steps):
        loss, dW = gradient_reg(X, y, W_learned, 2)
        print(loss)
        W_learned = W_learned - eta * dW
        
    return W_learned
    

In [167]:
W_star_reg = gradient_descent_reg(X_train, y_train, W, eta=0.001, steps=20)

54.20758211976921
51.076752717461495
48.764681892306726
45.150246857914915
41.20739764734916
37.842593970072826
34.40372589873632
30.297678814172144
26.947244821759636
23.67920299683294
21.65552984702191
20.159673247871645
18.719435368574114
17.37361158137452
16.031237446541766
14.79426200478653
13.69071268963535
12.341406359158503
11.25382383570278
9.978812350016465


In [168]:
for index, x in enumerate(X_test.dot(W_star_reg)):
    pred = x.argmax()
    true_label = y_test[index].argmax()
    print(pred, true_label)

3 3
5 5
5 5
4 4
0 0
0 0


If we look at the two different weight matrices (one regularised, the other not), we notice the following:

In [169]:
W_star[0:5]

array([[-5.78647135e-04,  1.28346720e-03, -1.93445596e-03,
         2.80640163e-03, -2.44218375e-04, -1.20728113e-03],
       [-7.96955403e-04,  7.22211518e-04, -3.49303253e-03,
        -3.51679433e-03,  5.01353061e-03,  2.24429131e-03],
       [-1.74911317e-03, -1.79858176e-03,  3.92019550e-03,
        -4.73300532e-03,  5.45948384e-03, -7.59202032e-04],
       [ 3.90212020e-03, -1.66440595e-03, -1.09472119e-03,
        -4.97761960e-04,  9.07647146e-05, -8.39547695e-04],
       [-1.87037704e-04, -1.13292487e-03,  9.40532198e-04,
        -5.72550923e-03,  7.10109402e-03, -9.25386999e-04]])

In [170]:
W_star_reg[0:5]

array([[-5.20655827e-04,  1.05672338e-03, -1.40708426e-03,
         2.39033138e-03, -4.62027423e-04, -9.33267750e-04],
       [-5.93592079e-04,  4.90021476e-04, -2.30015460e-03,
        -2.06140822e-03,  3.31698330e-03,  1.31967699e-03],
       [-1.52572058e-03, -1.39847125e-03,  2.97273629e-03,
        -3.91436072e-03,  4.52360052e-03, -3.21388872e-04],
       [ 3.38964849e-03, -1.44861974e-03, -9.56067745e-04,
        -6.55413811e-04,  7.09559278e-05, -5.03024378e-04],
       [-2.05402557e-04, -9.42166370e-04,  6.01204214e-04,
        -4.90855116e-03,  5.93320763e-03, -4.08228655e-04]])

## By the way ...
### In scikit-learn it's much easier to implement this :-)


In [88]:
from sklearn.svm import LinearSVC
clf = LinearSVC(random_state=0, multi_class='crammer_singer', loss='hinge')
clf.fit(X_train, train.y)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=1000,
     multi_class='crammer_singer', penalty='l2', random_state=0,
     tol=0.0001, verbose=0)

In [89]:
clf.predict(X_test)

array(['en', 'fi', 'fi', 'de', 'fr', 'fr'], dtype=object)

In [90]:
test.y

4     en
12    fi
16    fi
26    de
41    fr
42    fr
Name: y, dtype: object

We see that with our naive implementation, we do not much worse than with scikit's. scikit's implementation is of course much more elaborate and uses the vectorised operation and possibly other optimisation techniques in order to make its SVM (or SVC) better.